In [90]:
import pandas as pd
import altair as alt
import altair_latimes as lat
from pandas.tseries.offsets import MonthEnd

In [91]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [92]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Import

In [93]:
df = pd.read_csv("../data/raw/uw-usage.csv", parse_dates=["Reporting Month"])

In [95]:
baselines_df = pd.read_csv("../data/raw/uw-2020-baseline-values.csv")

### Clean

Remove junk from column names

In [96]:
df.columns = df.columns.str.replace(" ","_").str.replace("-","_").str.replace(r'[#,@,&,(,)]', '', regex=True).str.lower()

Eliminate double spaces in supplier names

In [97]:
df.supplier_name = df.supplier_name.str.replace("  ", " ")

Clean up baselines column names as we did in the previous notebook

In [98]:
baselines_df.columns = baselines_df.columns.str.replace(" ","_").str.replace("-","_").str.replace(r'[#,@,&,(,)]', '', regex=True).str.lower()

In [99]:
baselines_df.supplier_name = baselines_df.supplier_name.str.replace("  ", " ")

In [100]:
baselines_trimmed_df = baselines_df[[
    'supplier_name', 
    'public_water_system_id', 
    'month', 
    #'original_units',
    # 'total_potable_production_original_units',
    # 'potable_commercial_agriculture_original_units',
    'total_potable_production_minus_ag_gallons', 
    # 'staff_notes'
]].copy()

Get rid of some unnecessary columns

In [101]:
trim_df = df[[
    'supplier_name', 
    'public_water_system_id', 
    'reporting_month', 
    'county',
    'hydrologic_region', 
    'climate_zone', 
    'total_population_served',
    'calculated_total_potable_water_production_gallons_ag_excluded',
    'calculated_r_gpcd', 
    # 'month'
]].copy()

### Get populations by supplier, hydrologic region and state total

Using populations from the beginning of the drought emergency

In [102]:
pops_by_supplier = df[
    df.reporting_month == "2021-07-15"
][["supplier_name","hydrologic_region","total_population_served"]]

In [103]:
pops_by_region = pops_by_supplier.groupby(
    ["hydrologic_region"]
)["total_population_served"].sum().reset_index()

In [104]:
pop_state = pops_by_region.total_population_served.sum()
pop_state

37110227

### Merge

Filter df to July 2021 and later

In [105]:
filter_df = trim_df[trim_df.reporting_month >= "2021-07-15"].copy()

Remove -999999999

In [106]:
filter_df = filter_df[filter_df.calculated_total_potable_water_production_gallons_ag_excluded != -999999999]

Make a month column to merge with baselines

In [107]:
filter_df["month"] = filter_df["reporting_month"].dt.month

In [108]:
merge_df = pd.merge(
    filter_df, 
    baselines_trimmed_df, 
    how="left", 
    on=["supplier_name", "public_water_system_id", "month"]
).rename(
    columns={
        "calculated_total_potable_water_production_gallons_ag_excluded": "total_gallons_current",
        "total_potable_production_minus_ag_gallons": "total_gallons_baseline"
    }
).drop("month", axis=1)

### Remove duplicates

In [109]:
tmp = merge_df.set_index(['supplier_name', 'reporting_month'])

In [110]:
merge_df = tmp[~tmp.index.duplicated()].reset_index()

### Calculate percent changes by month

In [111]:
def pct_change(new, old):
    return (new - old) / old

By supplier

In [112]:
merge_df["gallons_pct_change"] = merge_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

Filter out outliers

In [113]:
# lower_thresh=-0.5
# upper_thresh=1.3

In [114]:
# districts_filtered = merge_df[
#     (merge_df['gallons_pct_change']<upper_thresh)&(merge_df['gallons_pct_change']>lower_thresh)
# ]

By region

In [115]:
regions_df = merge_df.groupby(
    ["hydrologic_region", "reporting_month"]
)[["total_gallons_current","total_gallons_baseline", "supplier_name"]].agg({
    "total_gallons_current":"sum",
    "total_gallons_baseline":"sum",
    "supplier_name":"size"
}).reset_index()

In [116]:
regions_df["gallons_pct_change"] = regions_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

Statewide

In [117]:
merge_df["state"] = "Statewide"

In [118]:
statewide_df = merge_df.groupby(["state", "reporting_month"])[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

In [119]:
statewide_df["gallons_pct_change"] = statewide_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

### Calculate cumulative savings since July 2021

Set up some vars and function

In [120]:
def month_diff(a, b):
    return 12 * (a.year - b.year) + (a.month - b.month)

In [121]:
def gallons_per_unit(gallons, baseline, population, time):
    return ((baseline - gallons) / population) / time

In [122]:
# voluntary_reductions_begin = pd.to_datetime('2021-07-08')
voluntary_reductions_begin = pd.to_datetime('2021-07-01')
voluntary_reductions_begin

Timestamp('2021-07-01 00:00:00')

In [123]:
latest_month = statewide_df[
    statewide_df.reporting_month == statewide_df.reporting_month.max()
].iloc[0]['reporting_month']

latest_month = pd.to_datetime(latest_month, format="%Y%m") + MonthEnd(0)

latest_month

Timestamp('2023-03-31 00:00:00')

In [124]:
months_since_reductions_begin = month_diff(latest_month, voluntary_reductions_begin)
months_since_reductions_begin

20

In [125]:
days_since_reductions = (latest_month - voluntary_reductions_begin).days
days_since_reductions

638

Get date of this monthly report

In [126]:
date = statewide_df[
    (statewide_df.reporting_month==statewide_df.reporting_month.max())
].iloc[0]["reporting_month"]

Drop suppliers with missing data

In [127]:
counts = merge_df.groupby("supplier_name")["reporting_month"].count().reset_index()

In [128]:
max_count = max(counts.reporting_month)
max_count

21

In [129]:
suppliers_with_complete_data = list(counts[counts.reporting_month == max_count].supplier_name)

In [130]:
complete_data = merge_df[merge_df.supplier_name.isin(suppliers_with_complete_data)].copy()

In [131]:
complete_data[complete_data.reporting_month == "2023-03-15"]

,supplier_name,reporting_month,public_water_system_id,county,hydrologic_region,climate_zone,total_population_served,total_gallons_current,calculated_r_gpcd,total_gallons_baseline,gallons_pct_change,state
0,East Bay Municipal Utilities District,2023-03-15,CA0110005,"Alameda,Contra Costa",San Francisco Bay,3,1440000,3672000000.000,47.710,4420700000.000,-0.169,Statewide
38,Yorba Linda Water District,2023-03-15,CA3010037,Orange,South Coast,8,83952,228920103.030,74.289,368889400.080,-0.379,Statewide
59,Long Beach City of,2023-03-15,CA1910065,Los Angeles,South Coast,6,460682,1032947670.000,55.694,1209560497.024,-0.146,Statewide
80,"Los Banos, City of",2023-03-15,CA2410005,Merced,San Joaquin River,12,46639,130395000.000,52.309,169832000.000,-0.232,Statewide
101,Turlock City of,2023-03-15,CA5010019,Stanislaus,San Joaquin River,12,74820,353480000.000,64.941,459400000.000,-0.231,Statewide
...,...,...,...,...,...,...,...,...,...,...,...,...
8281,"Corcoran, City of",2023-03-15,CA1610004,Kings,Tulare Lake,13,22535,78237000.000,50.914,102218000.000,-0.235,Statewide
8302,City of Newman Water Department,2023-03-15,CA5010013,Stanislaus,San Joaquin River,12,12244,35424354.000,57.864,46302000.000,-0.235,Statewide
8401,Greenfield County Water District,2023-03-15,CA1510024,Kern,Tulare Lake,4,11414,39829565.000,103.110,49023000.000,-0.188,Statewide
8442,Thermalito Water and Sewer District,2023-03-15,CA0410008,Butte,Sacramento River,11,12144,28285342.224,61.896,40275509.451,-0.298,Statewide


By district

In [132]:
district_cumulative_savings_df = complete_data.groupby(
    ["supplier_name","hydrologic_region"]
)[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

In [133]:
district_cumulative_savings_df = pd.merge(
    district_cumulative_savings_df,
    pops_by_supplier[["supplier_name","hydrologic_region","total_population_served"]],
    how="left",
    on=["supplier_name","hydrologic_region"]
)

In [134]:
district_cumulative_savings_df["cumulative_pct_change"] = district_cumulative_savings_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [135]:
district_cumulative_savings_df['date'] = date

In [136]:
district_cumulative_savings_df['months_since'] = months_since_reductions_begin
district_cumulative_savings_df['days_since'] = days_since_reductions

In [137]:
district_cumulative_savings_df['gallons_saved_per_month'] = district_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.months_since
    ), axis=1
)

In [138]:
district_cumulative_savings_df['gallons_saved_per_day'] = district_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.days_since
    ), axis=1
)

In [139]:
district_cumulative_savings_df.sort_values("gallons_saved_per_day", ascending=False)

,supplier_name,hydrologic_region,total_gallons_current,total_gallons_baseline,total_population_served,cumulative_pct_change,date,months_since,days_since,gallons_saved_per_month,gallons_saved_per_day
320,Vernon City of,South Coast,3471293961.510,3870557378.473,112,-0.103,2023-03-15,20,638,178242.597,5587.542
143,Humboldt Bay Municipal Water District,North Coast,102519000.000,174266000.000,602,-0.412,2023-03-15,20,638,5959.053,186.804
170,Livingston City of,San Joaquin River,3819717000.000,4834329000.000,15052,-0.210,2023-03-15,20,638,3370.356,105.654
50,California Water Service Company Westlake,South Coast,3208296360.900,4020289456.673,19489,-0.202,2023-03-15,20,638,2083.209,65.304
325,West Kern Water District,Tulare Lake,7804089089.370,8552860276.011,22113,-0.088,2023-03-15,20,638,1693.057,53.074
...,...,...,...,...,...,...,...,...,...,...,...
207,Oildale Mutual Water Company,Tulare Lake,5082317598.060,4955180289.703,34133,0.026,2023-03-15,20,638,-186.238,-5.838
69,Coachella City of,Colorado River,4247720000.000,4062500000.000,45635,0.046,2023-03-15,20,638,-202.936,-6.362
93,El Centro City of,Colorado River,4641609000.000,4268718000.000,46364,0.087,2023-03-15,20,638,-402.134,-12.606
96,El Segundo City of,South Coast,3948353853.412,3157502538.939,16654,0.250,2023-03-15,20,638,-2374.358,-74.431


In [140]:
chart_data = complete_data[complete_data.supplier_name.str.contains("El Segundo")]
name = chart_data.iloc[0]["supplier_name"]

alt.Chart(
    chart_data
).mark_bar().encode(
    x=alt.X('yearmonth(reporting_month):T', axis=alt.Axis(title="Reporting month")),
    y=alt.Y("gallons_pct_change:Q", axis=alt.Axis(format="%", title="Percent change in gallons used")),
    color=alt.condition(
        alt.datum.gallons_pct_change > 0,
        alt.value("#e6ae56"),  # The positive color
        alt.value("#83c6e0")  # The negative color
    ),
    tooltip=["reporting_month","gallons_pct_change"]
).properties(title=f"Monthly water conservation in {name}", width=600)

alt.Chart(...)

In [141]:
df.iloc[36934]["qualification"]

nan

In [142]:
df.iloc[18213]['qualification']

nan

In [143]:
df[
    (df.supplier_name=="Paradise Irrigation District")&
    (df.reporting_month >= "2021-07-15")
]

,supplier_name,public_water_system_id,reporting_month,county,hydrologic_region,climate_zone,total_population_served,reference_2014_population,county_under_drought_declaration,water_shortage_contingency_stage_invoked,...,reported_preliminary_commercial_industrial_and_institutional_water,reported_final_commercial_industrial_and_institutional_water,reported_recycled_water,reported_non_revenue_water,calculated_total_potable_water_production_gallons_ag_excluded,calculated_total_potable_water_production_gallons_2013_ag_excluded,calculated_commercial_agricultural_water_gallons,calculated_commercial_agricultural_water_gallons_2013,calculated_r_gpcd,qualification
18387,Paradise Irrigation District,CA0410007,2023-03-15,Butte,Sacramento River,11,8800,26032,Yes,1,...,0.000,0.000,0.000,0.000,72600000.000,101600000.000,0.000,0.000,25.689,still working on meters to all customers.
18388,Paradise Irrigation District,CA0410007,2023-02-15,Butte,Sacramento River,11,8800,26032,Yes,1,...,0.000,0.000,0.000,0.000,69300000.000,76100000.000,0.000,0.000,26.883,Still working on meter system.
18389,Paradise Irrigation District,CA0410007,2023-01-15,Butte,Sacramento River,11,8000,26032,Yes,0,...,0.000,0.000,0.000,0.000,74900000.000,82500000.000,0.000,0.000,29.032,Still working on meters.
18390,Paradise Irrigation District,CA0410007,2022-12-15,Butte,Sacramento River,11,8000,26032,Yes,1,...,0.000,0.000,0.000,0.000,78100000.000,102300000.000,0.000,0.000,30.194,NaN
18391,Paradise Irrigation District,CA0410007,2022-11-15,Butte,Sacramento River,11,8000,26032,Yes,1,...,0.000,0.000,0.000,0.000,82800000.000,117400000.000,0.000,0.000,33.200,Meter project still underway. Majority should ...
18392,Paradise Irrigation District,CA0410007,2022-10-15,Butte,Sacramento River,11,8000,26032,Yes,1,...,0.000,0.000,0.000,0.000,131600000.000,170700000.000,0.000,0.000,51.097,Meter project still underway. Majority should ...
18393,Paradise Irrigation District,CA0410007,2022-09-15,Butte,Sacramento River,11,8000,26032,Yes,1,...,0.000,0.000,0.000,0.000,144400000.000,230100000.000,0.000,0.000,57.600,Still in recovery mode from fire.
18394,Paradise Irrigation District,CA0410007,2022-08-15,Butte,Sacramento River,11,8000,26032,Yes,1,...,0.000,0.000,0.000,0.000,174200000.000,309900000.000,0.000,0.000,67.355,NaN
18395,Paradise Irrigation District,CA0410007,2022-07-15,Butte,Sacramento River,11,8000,26032,Yes,We haven't,...,0.000,0.000,0.000,0.000,162400000.000,327500000.000,0.000,0.000,62.710,Still working on the metering system.
18396,Paradise Irrigation District,CA0410007,2022-06-15,Butte,Sacramento River,11,6000,26032,Yes,We haven't,...,0.000,0.000,0.000,0.000,122600000.000,276200000.000,0.000,0.000,65.600,Still working on meters to homes.


In [144]:
complete_data[
    (complete_data.supplier_name=="El Segundo City of")&
    (complete_data.reporting_month >= "2021-07-15") &
    (complete_data.reporting_month != "2021-08-15")
].groupby("supplier_name")[["total_gallons_current","total_gallons_baseline"]].sum().reset_index()

,supplier_name,total_gallons_current,total_gallons_baseline
0,El Segundo City of,3622159794.396,2967898033.346


In [145]:
( (3476540999.868-2817199764.284) / 16654) / 607

65.22333272305075

In [146]:
chart_data = complete_data[complete_data.supplier_name == "Los Angeles Department of Water and Power"]
name = chart_data.iloc[0]["supplier_name"]

alt.Chart(
    chart_data
).mark_bar().encode(
    x=alt.X('yearmonth(reporting_month):T', axis=alt.Axis(title="Reporting month")),
    y=alt.Y("gallons_pct_change:Q", axis=alt.Axis(format="%", title="Percent change in gallons used")),
    color=alt.condition(
        alt.datum.gallons_pct_change > 0,
        alt.value("#e6ae56"),  # The positive color
        alt.value("#83c6e0")  # The negative color
    ),
    tooltip=["reporting_month","gallons_pct_change"]
).properties(title=f"Monthly water conservation in {name}", width=600)

alt.Chart(...)

In [147]:
alt.Chart(
    complete_data[complete_data.supplier_name.str.contains("El Segundo")]
).mark_bar().encode(
    x=alt.X('yearmonth(reporting_month):T'),
    y="total_gallons_current",
    tooltip=["reporting_month"]
).properties(width=600)

alt.Chart(...)

Drop districts for which we can't calculate a percentage change

In [148]:
drop_na_districts = district_cumulative_savings_df.dropna(subset="cumulative_pct_change")

By region

In [149]:
regions_cumulative_savings_df = district_cumulative_savings_df.groupby(
    ["hydrologic_region"]
)[
    ["total_gallons_current","total_gallons_baseline","supplier_name"]
].agg(
    {"total_gallons_current":"sum","total_gallons_baseline":"sum","supplier_name":"size"}
).reset_index().rename(
    columns={"supplier_name": "total_reports"}
)

In [150]:
regions_cumulative_savings_df = pd.merge(
    regions_cumulative_savings_df,
    pops_by_region[["hydrologic_region","total_population_served"]],
    how="left",
    on="hydrologic_region"
)

In [151]:
regions_cumulative_savings_df["cumulative_pct_change"] = regions_cumulative_savings_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [152]:
regions_cumulative_savings_df['date'] = date

In [153]:
regions_cumulative_savings_df['months_since'] = months_since_reductions_begin
regions_cumulative_savings_df['days_since'] = days_since_reductions

In [154]:
regions_cumulative_savings_df['gallons_saved_per_month'] = regions_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.months_since
    ), axis=1
)

In [155]:
regions_cumulative_savings_df['gallons_saved_per_day'] = regions_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.days_since
    ), axis=1
)

In [156]:
regions_cumulative_savings_df

,hydrologic_region,total_gallons_current,total_gallons_baseline,total_reports,total_population_served,cumulative_pct_change,date,months_since,days_since,gallons_saved_per_month,gallons_saved_per_day
0,Central Coast,69234811310.229,73365423757.949,25,1293270,-0.056,2023-03-15,20,638,159.696,5.006
1,Colorado River,112284871247.764,115008590870.304,12,780308,-0.024,2023-03-15,20,638,174.528,5.471
2,North Coast,22283199241.799,25925047982.229,14,414740,-0.140,2023-03-15,20,638,439.052,13.763
3,North Lahontan,7288338736.000,7922874968.000,4,103768,-0.080,2023-03-15,20,638,305.748,9.585
4,Sacramento River,288271664942.155,314840012362.537,39,2843842,-0.084,2023-03-15,20,638,467.121,14.643
5,San Francisco Bay,387008527009.307,439153980464.404,39,6671142,-0.119,2023-03-15,20,638,390.829,12.252
6,San Joaquin River,151971956648.874,161235222659.375,23,1706054,-0.057,2023-03-15,20,638,271.482,8.510
7,South Coast,1437785153127.757,1531408638988.147,143,20450766,-0.061,2023-03-15,20,638,228.900,7.176
8,South Lahontan,66377056982.152,71314821747.303,13,863361,-0.069,2023-03-15,20,638,285.962,8.964
9,Tulare Lake,218989689443.312,228780298585.595,24,1982976,-0.043,2023-03-15,20,638,246.867,7.739


In [157]:
chart_data = regions_df[regions_df.hydrologic_region=="South Coast"]
name = chart_data.iloc[0]["hydrologic_region"]

alt.Chart(
    chart_data
).mark_bar().encode(
    x=alt.X('yearmonth(reporting_month):T'),
    y="gallons_pct_change:Q",
    color=alt.condition(
        alt.datum.gallons_pct_change > 0,
        alt.value("#e6ae56"),  # The positive color
        alt.value("#83c6e0")  # The negative color
    ),
    tooltip=["reporting_month","gallons_pct_change"]
).properties(title=f"Monthly water conservation in {name}", width=600)

alt.Chart(...)

Statewide

In [158]:
district_cumulative_savings_df["state"] = "California"

In [159]:
statewide_cumulative_savings_df = district_cumulative_savings_df.groupby(
    ["state"]
)[
    ["total_gallons_current","total_gallons_baseline", "supplier_name"]
].agg({
    "total_gallons_current": "sum",
    "total_gallons_baseline": "sum",
    "supplier_name": "size"
}).reset_index().rename(
    columns={"supplier_name": "total_reports"}
)

In [160]:
#statewide_cumulative_savings_df["total_reports"] = statewide_cumulative_savings_df["total_reports"]

In [161]:
statewide_cumulative_savings_df["total_population_served"] = pop_state

In [162]:
statewide_cumulative_savings_df["cumulative_pct_change"] = statewide_cumulative_savings_df.apply(lambda x: pct_change(x.total_gallons_current, x.total_gallons_baseline), axis=1)

In [163]:
statewide_cumulative_savings_df['date'] = date

In [164]:
statewide_cumulative_savings_df['months_since'] = months_since_reductions_begin
statewide_cumulative_savings_df['days_since'] = days_since_reductions

In [165]:
statewide_cumulative_savings_df['gallons_saved_per_month'] = regions_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.months_since
    ), axis=1
)

In [166]:
statewide_cumulative_savings_df['gallons_saved_per_day'] = statewide_cumulative_savings_df.apply(
    lambda x: gallons_per_unit(
        x.total_gallons_current, x.total_gallons_baseline, x.total_population_served, x.days_since
    ), axis=1
)

In [167]:
statewide_cumulative_savings_df

,state,total_gallons_current,total_gallons_baseline,total_reports,total_population_served,cumulative_pct_change,date,months_since,days_since,gallons_saved_per_month,gallons_saved_per_day
0,California,2761495268689.350,2968954912385.844,336,37110227,-0.070,2023-03-15,20,638,159.696,8.762


In [168]:
alt.Chart(
    statewide_df
).mark_bar().encode(
    x=alt.X('yearmonth(reporting_month):T', axis=alt.Axis(title="Reporting month")),
    y=alt.Y("gallons_pct_change:Q", axis=alt.Axis(format="%", title="Percent change in gallons used")),
    color=alt.condition(
        alt.datum.gallons_pct_change > 0,
        alt.value("#e6ae56"),  # The positive color
        alt.value("#83c6e0")  # The negative color
    ),
    tooltip=["reporting_month","gallons_pct_change"]
).properties(title=f"Monthly water conservation in California", width=600)

alt.Chart(...)

### Export